<a href="https://colab.research.google.com/github/8ahmedanwer8/rcCarIMUDataClassification/blob/main/imudataClassificiation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
from google.colab import drive
import pandas  as pd
import numpy as np 
import pickle
import matplotlib.pyplot as plt
drive.mount('/content/drive')
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Augmented_dataset.csv')
df.shape


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


(9999, 1806)

In [34]:

df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Augmented_dataset.csv')
#shape is 9999x1806. 9999 samples of accel and gyro data. each sample is 1806 columns long
#one accel/gyro data is [[x1,x2...,xn],[y1,y2...,yn],[z1,z2...,zn]] where n is equal to 300
#so we have two arrays each with 1 array which has 3 arrays which has 300 data points

#creating a proper column headers, probably there is a better way to do this but below works
col_names = ["ignore", "ignore"]
for i in range(1,301):
  name = "accelX " + str(i)
  col_names.append(name)
for i in range(1,301):
  name = "accelY " + str(i)
  col_names.append(name)
col_names.append("ignore")
for i in range(1,301):
  name = "accelZ " + str(i)
  col_names.append(name)
col_names.append("ignore")
for i in range(1,301):
  name = "gyroX " + str(i)
  col_names.append(name)
for i in range(1,301):
  name = "gyroY " + str(i)
  col_names.append(name)
for i in range(1,301):
  name = "gyroZ " + str(i)
  col_names.append(name)
col_names.append("ignore")
col_names.append("Labels")

df.columns = col_names

for index, row in df.iterrows():#remove poorly formatted rows, about 181 of these exist
  if pd.isna(row[1805]):
    df.drop(index, inplace=True)


df.iloc[:,1805]= df.iloc[:,1805].apply(lambda x:x.replace('}"', '') if pd.isna(x)!= True else x) #remove }" in the true labels column
df.iloc[:,1805]= df.iloc[:,1805].astype(int)#turn the labels from strings to integers (int64 to be specific)

df.iloc[:,1803]= df.iloc[:,1803].apply(lambda x:x.replace(']]),', '') if pd.isna(x)!= True else x) #remove ]]), in gyro{x,y,z} closing bracket
df.iloc[:,902]= df.iloc[:,902].apply(lambda x:x.replace(']]),', '') if pd.isna(x)!= True else x) #remove ]]), in accel{x,y,z} closing bracket
df.iloc[:,904]= df.iloc[:,904].apply(lambda x:x.replace('array([[', '') if pd.isna(x)!= True else x) #remove array([[, in gyro{x,y,z} opening bracket

df.iloc[:,301]= df.iloc[:,301].apply(lambda x:x.replace(']', '') if pd.isna(x)!= True else x) #remove ] from closing bracket of accelX  
df.iloc[:,601]= df.iloc[:,601].apply(lambda x:x.replace(']', '') if pd.isna(x)!= True else x) #remove ] from closing bracket of accelY  
df.iloc[:,302]= df.iloc[:,302].apply(lambda x:x.replace('[', '') if pd.isna(x)!= True else x) #remove [ from opening bracket of accelY
df.iloc[:,1203]= df.iloc[:,1203].apply(lambda x:x.replace(']', '') if pd.isna(x)!= True else x) #remove ] from closing bracket of gyroX
df.iloc[:,1204]= df.iloc[:,1204].apply(lambda x:x.replace('[', '') if pd.isna(x)!= True else x) #remove [ from opening bracket of gyroY
df.iloc[:,1503]= df.iloc[:,1503].apply(lambda x:x.replace(']', '') if pd.isna(x)!= True else x) #remove ] from closing bracket of gyroY
df.iloc[:,1504]= df.iloc[:,1504].apply(lambda x:x.replace('[', '') if pd.isna(x)!= True else x) #remove [ from opening bracket of gyroZ


df = df.drop(df.columns[[0,1,602,903,1804]], axis=1) #remove certain columns ("{'Acc':, array([[,[ for accelZ, 'Gyr': and 'Label':)
df.replace(to_replace=",", value="", regex = True, inplace = True) #remove the comma after each number in the df

# df.to_csv('processed_Augmented_dataset.csv', index=False)
df.sample(10)


,accelX 1,accelX 2,accelX 3,accelX 4,accelX 5,accelX 6,accelX 7,accelX 8,accelX 9,accelX 10,accelX 11,accelX 12,accelX 13,accelX 14,accelX 15,accelX 16,accelX 17,accelX 18,accelX 19,accelX 20,accelX 21,accelX 22,accelX 23,accelX 24,accelX 25,accelX 26,accelX 27,accelX 28,accelX 29,accelX 30,accelX 31,accelX 32,accelX 33,accelX 34,accelX 35,accelX 36,accelX 37,accelX 38,accelX 39,accelX 40,...,gyroZ 262,gyroZ 263,gyroZ 264,gyroZ 265,gyroZ 266,gyroZ 267,gyroZ 268,gyroZ 269,gyroZ 270,gyroZ 271,gyroZ 272,gyroZ 273,gyroZ 274,gyroZ 275,gyroZ 276,gyroZ 277,gyroZ 278,gyroZ 279,gyroZ 280,gyroZ 281,gyroZ 282,gyroZ 283,gyroZ 284,gyroZ 285,gyroZ 286,gyroZ 287,gyroZ 288,gyroZ 289,gyroZ 290,gyroZ 291,gyroZ 292,gyroZ 293,gyroZ 294,gyroZ 295,gyroZ 296,gyroZ 297,gyroZ 298,gyroZ 299,gyroZ 300,Labels
6770,6.68584184e-02,4.68590126e-02,2.65805544e-02,7.57112338e-03,-8.62120077e-03,-1.90528971e-02,-1.49692994e-02,7.96988503e-03,2.89211041e-02,2.67338280e-02,7.92223467e-03,-1.30646266e-02,-1.98348990e-02,-2.31190372e-03,1.86492367e-02,2.08915092e-02,1.64456341e-02,2.35487109e-02,4.33715873e-02,6.77182677e-02,6.74947991e-02,2.00516142e-02,-3.50874059e-02,-5.16498275e-02,-4.45223926e-02,-3.99447807e-02,-2.48779338e-02,1.90583508e-02,7.30741337e-02,1.08179230e-01,1.05208997e-01,5.54096195e-02,-1.11642433e-02,-5.63963974e-02,-6.99558060e-02,-5.34948501e-02,-2.23692401e-02,6.39599314e-03,3.38318021e-02,6.38343421e-02,...,5.78133251e+01,5.75688286e+01,5.70860181e+01,5.65525758e+01,5.63728615e+01,5.67101524e+01,5.69330252e+01,5.63916590e+01,5.47922650e+01,5.22287106e+01,4.97535572e+01,4.82899768e+01,4.78346634e+01,4.80790772e+01,4.83114047e+01,4.80021980e+01,4.74396655e+01,4.69643332e+01,4.65698644e+01,4.61523745e+01,4.55159726e+01,4.45100147e+01,4.31977273e+01,4.16829434e+01,4.00559259e+01,3.83617438e+01,3.64984425e+01,3.44916871e+01,3.29327989e+01,3.23263671e+01,3.24099465e+01,3.26812758e+01,3.23929088e+01,3.10223440e+01,2.91070980e+01,2.73295270e+01,2.59699909e+01,2.52123172e+01,2.52403333e+01,4
3954,5.42968750e-02,1.44230160e-01,1.31138943e-01,5.55544050e-02,1.11419582e-01,7.65098857e-02,1.75376592e-02,2.33387215e-02,8.03725050e-02,1.05912067e-01,2.26477030e-02,9.02409038e-03,4.29308930e-02,5.22460818e-02,1.73149794e-01,1.82356083e-01,-2.64215620e-02,-5.11937209e-02,2.21836188e-02,4.45306465e-02,1.27780010e-01,1.35691073e-01,7.31230184e-03,-8.93238754e-03,5.54351812e-02,6.56254072e-02,9.10240453e-02,2.26948823e-02,3.28754294e-02,2.74162350e-02,7.83814823e-02,-5.53803450e-03,5.14711078e-02,8.88723259e-02,8.77980597e-02,-1.62765933e-03,1.30332722e-02,8.67795697e-02,1.01051361e-01,1.83622107e-02,...,-3.89326248e+00,-2.15860954e+00,-2.56461995e+00,-2.59701727e-02,2.77272268e+00,3.50752530e+00,1.24873047e+00,-4.78482264e-01,-1.60131956e+00,-1.09021113e+00,4.09244699e-01,-9.56884450e-01,-4.25067115e+00,-1.82232942e+00,-1.23255098e+00,-1.47544067e+00,-1.21178765e+00,6.44185111e-01,9.78014763e-01,-3.14731179e+00,-3.90861839e+00,-5.22455449e+00,-5.76190828e+00,-2.84677888e+00,-4.36545288e+00,-9.06461672e+00,-6.61598926e+00,-5.25890357e+00,-3.47822730e+00,-1.84565100e+00,-8.41342822e-01,-1.65671743e+00,-8.38190639e-01,-1.96525390e+00,-4.17935682e+00,-3.75051968e+00,-1.45274258e+00,-1.06342367e+00,6.34765625e-01,3
8947,7.51138017e-02,1.09450026e-01,7.20699322e-02,1.35977281e-02,-1.53423784e-02,1.69443471e-02,6.33231476e-02,6.98226559e-02,4.06009302e-02,-3.25208197e-03,-3.41640564e-02,-2.97159682e-02,1.94329237e-03,4.32773281e-02,7.25187684e-02,7.40410521e-02,6.59999440e-02,7.00278935e-02,7.84094727e-02,7.77293660e-02,6.85549541e-02,5.76014296e-02,6.11793100e-02,8.69187263e-02,8.44501144e-02,1.20749427e-02,-3.55733064e-02,4.00527217e-02,1.32424564e-01,1.06485309e-01,2.87166609e-02,8.11497382e-03,2.29167171e-02,2.09256064e-02,2.55625668e-02,7.02801743e-02,9.82935428e-02,4.42408316e-02,-2.42497024e-02,-1.50704302e-02,...,1.31055917e+00,2.58073446e+00,3.19001050e+00,2.24309278e+00,1.02884840e+00,6.47116752e-01,5.78260357e-01,3.43935696e-01,4.19588431e-01,9.49026312e

In [37]:
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/processed_Augmented_dataset.csv')
print(df.iloc[:,1800].value_counts()) #there are 5 labels and about 2000 samples for each label so data is well-distributed

#create numpy arrays of the data just to test the model at the end 
predX = [] 
predY = []
predY = df['Labels']
predX = df.iloc[0:,0:1800]
predX = np.array(predX)
predY= np.array(predY)




2    1993
5    1971
4    1964
3    1955
1    1935
Name: Labels, dtype: int64


In [38]:
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.utils import to_categorical

#creating numpy arrays of the data (X and y) which will be used for the deep neural network
df = df.sample(frac = 1) #shuffle the data
X = []
y = []
y = df['Labels']
X = df.iloc[0:,0:1800]
y = np.array(y)
X = np.array(X)
y = to_categorical(y) #one hot encode the labels


In [41]:
#deep neural network
model = Sequential()
model.add(Dense(512, input_shape = X.shape[1:], activation = 'relu'))
model.add(Dense(256, activation = 'relu'))
model.add(Dense(128, activation = 'relu'))
model.add(Dense(64, activation = 'relu'))
model.add(Dense(6,activation = 'softmax')) #6 is the number of output neurons because it should match the shape of y

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics = ['accuracy'])
model.fit(X,y, epochs =20, batch_size = 32, validation_split = 0.4) 



Epoch 1/20
185/185 [==============================] - 3s 14ms/step - loss: 1.1497 - accuracy: 0.8143 - val_loss: 0.2357 - val_accuracy: 0.9127
Epoch 2/20
185/185 [==============================] - 2s 13ms/step - loss: 0.7602 - accuracy: 0.9212 - val_loss: 0.3205 - val_accuracy: 0.9241
Epoch 3/20
185/185 [==============================] - 2s 13ms/step - loss: 0.5240 - accuracy: 0.9640 - val_loss: 0.6394 - val_accuracy: 0.8931
Epoch 4/20
185/185 [==============================] - 2s 13ms/step - loss: 0.0989 - accuracy: 0.9727 - val_loss: 0.1847 - val_accuracy: 0.9435
Epoch 5/20
185/185 [==============================] - 2s 13ms/step - loss: 0.0140 - accuracy: 0.9964 - val_loss: 0.2272 - val_accuracy: 0.9427
Epoch 6/20
185/185 [==============================] - 2s 13ms/step - loss: 0.0031 - accuracy: 0.9997 - val_loss: 0.2147 - val_accuracy: 0.9440
Epoch 7/20
185/185 [==============================] - 2s 12ms/step - loss: 5.6146e-04 - accuracy: 1.0000 - val_loss: 0.2269 - val_accuracy: 0.

In [42]:
import tensorflow
model.save('rcCar.model')
myModel = tensorflow.keras.models.load_model('rcCar.model')

INFO:tensorflow:Assets written to: rcCar.model/assets


INFO:tensorflow:Assets written to: rcCar.model/assets


In [43]:
pred = myModel.predict([predX])#outputs the probability distribution for each data sample (X1, X2, X3...)
ctr = 0

one = 'acceleration'
two = 'normal driving'
three = 'collision'
four = 'left turn'
five = 'right turn'

for i in range(1,9818):
  answer = np.argmax(pred[i]) #finds the highest probability from each distribution and that is our model's prediction
  
  if answer == 1: message = one
  if answer == 2: message = two
  if answer == 3: message = three
  if answer == 4: message = four
  if answer == 5: message = five

  print(i, answer," ", message, "\t", predY[i])
  if (answer != predY[i]):
    print("found a mistake!")
    ctr = ctr +1
print("\nThe amount of incorrect predictions were ", ctr)
print((ctr/9818)*100, "% were incorrect")





Streaming output truncated to the last 5000 lines.
found a mistake!
5021 1   acceleration 	 1
5022 3   collision 	 1
found a mistake!
5023 1   acceleration 	 1
5024 1   acceleration 	 1
5025 1   acceleration 	 1
5026 1   acceleration 	 1
5027 1   acceleration 	 1
5028 1   acceleration 	 1
5029 1   acceleration 	 1
5030 1   acceleration 	 1
5031 1   acceleration 	 1
5032 1   acceleration 	 1
5033 1   acceleration 	 1
5034 3   collision 	 1
found a mistake!
5035 1   acceleration 	 1
5036 1   acceleration 	 1
5037 1   acceleration 	 1
5038 1   acceleration 	 1
5039 1   acceleration 	 1
5040 1   acceleration 	 1
5041 1   acceleration 	 1
5042 1   acceleration 	 1
5043 1   acceleration 	 1
5044 1   acceleration 	 1
5045 1   acceleration 	 1
5046 1   acceleration 	 1
5047 3   collision 	 1
found a mistake!
5048 3   collision 	 1
found a mistake!
5049 1   acceleration 	 1
5050 1   acceleration 	 1
5051 1   acceleration 	 1
5052 1   acceleration 	 1
5053 1   acceleration 	 1
5054 1   accelerat